In [1]:
import keras
import numpy as np
import pandas as pd
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 2118/2118 [00:00<00:00, 6770.27it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 18841/18841 [00:05<00:00, 3242.42it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1614976/1614976 [00:00<00:00, 2863004.74it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2118/2118 [00:01<00:00, 1412.50it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 18841/18841 [00:44<00:00, 421.13it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 122/122 [00:00<00:00, 1604.91it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 1115/1115 [00:02<00:00, 438.25it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 237/237 [00:00<00:00, 1638.25it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit => SumRepresentUnit: 100%|██████████| 2300/2300 [00:05<00:00, 427.10it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DSSMModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DSSMModel.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 9644)         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 9644)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 300)          2893500     text_left[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 300)          2893500     text_right[0][0]                 
__________________________________________________________________________

In [7]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [8]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20, shuffle=True)
len(train_generator)

102

In [9]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=5, use_multiprocessing=False)

Epoch 1/30
102/102 [==============================] - 8s 75ms/step - loss: 0.9976
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.433040 - normalized_discounted_cumulative_gain@5(0):0.507081 - mean_average_precision(0):0.465983
Epoch 2/30
102/102 [==============================] - 7s 65ms/step - loss: 0.9881
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.433040 - normalized_discounted_cumulative_gain@5(0):0.507081 - mean_average_precision(0):0.465983
Epoch 3/30
102/102 [==============================] - 7s 64ms/step - loss: 0.9729
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.433040 - normalized_discounted_cumulative_gain@5(0):0.505449 - mean_average_precision(0):0.465842
Epoch 4/30
102/102 [==============================] - 7s 64ms/step - loss: 0.9661
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.434145 - normalized_discounted_cumulative_gain@5(0):0.505738 - mean_average_precision(0):0.467495
Epoch 5/